# Homework 2
### Modern Data Structures
### Hafsah Shaik
### Wednesday, September 27, 2023

In [1]:
# Import necessary libraries
import pandas as pd

# Load data
df = pd.read_csv('/Users/hafsahshaik/Desktop/U.S._Chronic_Disease_Indicators__CDI_.csv')

# Add the path to your .gitignore
with open('.gitignore', 'a') as f:
   f.write('path_to_U.S._Chronic_Disease_Indicators_CDI_.csv\n')

df.head()

/var/folders/0h/84sw5vkn1m9_l70_9hn00r440000gn/T/ipykernel_76904/4204124084.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/hafsahshaik/Desktop/U.S._Chronic_Disease_Indicators__CDI_.csv')


,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2014,2014,AR,Arkansas,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,5,AST,AST3_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN
1,2018,2018,CO,Colorado,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,8,AST,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2018,2018,DC,District of Columbia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,11,AST,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2017,2017,GA,Georgia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,13,AST,AST3_1,NMBR,GENDER,GENF,NaN,NaN,NaN,NaN
4,2010,2010,MI,Michigan,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,26,AST,AST3_1,NMBR,RACE,HIS,NaN,NaN,NaN,NaN


## Selection of Data and Reshaping the Data

### 1. Remove all columns you do not need.
Remove all columns you do not need for the analysis (All done in Python, of course. No Excel acrobatics.). We are interested in two sets of variables. Select the following variables and remove all others:
a) Binge Drinking: Binge drinking prevalence among adults aged >= 18 years, Crude Prevalence in Percent. We would like to obtain this variable for the overall population, as well separately for females and males.
b) Poverty: Poverty, Crude Prevalence in Percent. We only want the overall poverty prevalence to make things a bit easier.

In [2]:
# Define the conditions for binge drinking and poverty
mask_binge = (
    (df['Question'] == 'Binge drinking prevalence among adults aged >= 18 years') &
    (df['DataValueType'] == 'Crude Prevalence') &
    ((df['Stratification1'] == 'Overall') | 
     (df['Stratification1'] == 'Female') |
     (df['Stratification1'] == 'Male'))
)
mask_poverty = (
    (df['Question'] == 'Poverty, Crude Prevalence in Percent') &
    (df['Stratification1'] == 'Overall')
)

# Filter rows based on the defined conditions
df_filtered = df[mask_binge | mask_poverty]

# Keep necessary columns for analysis
columns_to_keep = ['LocationDesc', 'LocationAbbr', 'YearStart', 'DataValue', 'Stratification1']
df_cleaned = df_filtered[columns_to_keep]

df_cleaned.head()

,LocationDesc,LocationAbbr,YearStart,DataValue,Stratification1
529731,Alaska,AK,2021,17.5,Overall
530247,Alaska,AK,2019,11.1,Female
530534,Alabama,AL,2012,12.3,Overall
530605,Alabama,AL,2014,12.0,Overall
531039,Alaska,AK,2011,20.8,Overall


### 2. Convert the dataset to a wide format.
Convert the dataset to a wide format data set using the commands from the pandas package.

In [3]:
# Convert to wide format
df_wide = df_cleaned.pivot_table(
    index=['LocationDesc', 'LocationAbbr', 'YearStart'],
    columns='Stratification1',
    values='DataValue',
    aggfunc='first'
).reset_index()

# Remove the name from columns
df_wide.columns.name = None  

In [4]:
df_wide

,LocationDesc,LocationAbbr,YearStart,Female,Male,Overall
0,Alabama,AL,2011,7.9,20.2,13.7
1,Alabama,AL,2012,7.0,18.2,12.3
2,Alabama,AL,2013,6.5,16.3,11.2
3,Alabama,AL,2014,6.8,17.9,12.0
4,Alabama,AL,2015,7.5,16.2,11.6
...,...,...,...,...,...,...
589,Wyoming,WY,2017,13.1,22.8,18.0
590,Wyoming,WY,2018,10.0,24.6,17.4
591,Wyoming,WY,2019,12.9,22.4,17.7
592,Wyoming,WY,2020,12.0,20.5,16.4


### 3. Rename the variables to follow the format.

Rename the variables to follow the format below.
Your dataset should now be in a wide state-year format with the following variables:

state: Name of the State
stateabb: State Abbreviation
year: year of observation
binge_all: Binge drinking prevalence among all adults aged >= 18 years
binge_male: Binge drinking prevalence among male adults aged >= 18 years
binge_female: Binge drinking prevalence among female adults aged >= 18 years
poverty: Poverty, Crude Prevalence in Percent
Provide an overview of the dataset by printing its size (using the shape command) and some summary statistics (using the describe command).

Save the cleaned dataset as binge_clean.csv. That file should be included in the uploaded files for your homework submission.

In [10]:
# Rename variables
df_wide.rename(columns={
    'LocationDesc': 'state',
    'LocationAbbr': 'stateabb',
    'YearStart': 'year',
    'Overall': 'binge_all',
    'Female': 'binge_female',
    'Male': 'binge_male'
}, inplace=True)

# Drop the 'binge_all' from the poverty data rows
mask_poverty_rows = df_wide['binge_female'].isna() & df_wide['binge_male'].isna()
df_wide.loc[mask_poverty_rows, 'binge_all'] = None

# Rename binge_all column to poverty for the rows representing poverty data
df_wide['poverty'] = df_wide.loc[mask_poverty_rows, 'binge_all']

# Print the overview
print(df_wide.shape)
print(df_wide.describe())

(594, 7)
              year   binge_all  poverty
count   594.000000  594.000000      0.0
mean   2015.994949   16.751515      NaN
std       3.162540    3.066388      NaN
min    2011.000000    9.600000      NaN
25%    2013.000000   14.825000      NaN
50%    2016.000000   16.600000      NaN
75%    2019.000000   18.400000      NaN
max    2021.000000   27.200000      NaN


In [11]:
df_wide

,state,stateabb,year,binge_female,binge_male,binge_all,poverty
0,Alabama,AL,2011,7.9,20.2,13.7,NaN
1,Alabama,AL,2012,7.0,18.2,12.3,NaN
2,Alabama,AL,2013,6.5,16.3,11.2,NaN
3,Alabama,AL,2014,6.8,17.9,12.0,NaN
4,Alabama,AL,2015,7.5,16.2,11.6,NaN
...,...,...,...,...,...,...,...
589,Wyoming,WY,2017,13.1,22.8,18.0,NaN
590,Wyoming,WY,2018,10.0,24.6,17.4,NaN
591,Wyoming,WY,2019,12.9,22.4,17.7,NaN
592,Wyoming,WY,2020,12.0,20.5,16.4,NaN


In [12]:
# Save the cleaned dataset
df_wide.to_csv('binge_clean.csv', index=False)

## Data Transformation and Summary Results

### 4. Produce a table that shows the overall prevalences.
Produce a table that shows the overall, female, and male binge drinking prevalences across U.S. States in the most recent year of data for the Top 10 binge drinking states (i.e. the ones with the highest prevalence in the overall population). Use the relevant pandas commands to select the right variables, sort the data, and filter the data frame.

In [13]:
# Convert the 'binge_all' column to a numeric type
df_wide['binge_all'] = pd.to_numeric(df_wide['binge_all'], errors='coerce')

# Get most recent year and filter dataset
most_recent_year = df_wide['year'].max()
df_recent = df_wide[df_wide['year'] == most_recent_year]

# Get top 10 states
top_10_states = df_recent.nlargest(10, 'binge_all')

# Select desired columns and display the table
top_10_states = top_10_states[['state', 'stateabb', 'binge_all', 'binge_female', 'binge_male']]
top_10_states

,state,stateabb,binge_all,binge_female,binge_male
98,District of Columbia,DC,22.5,18.8,26.7
582,Wisconsin,WI,21.9,17.9,25.9
393,North Dakota,ND,20.8,15.0,26.3
306,Montana,MT,20.5,15.4,25.6
185,Iowa,IA,20.1,14.3,26.1
481,South Dakota,SD,19.6,14.8,24.4
317,Nebraska,NE,19.5,14.3,24.8
65,Colorado,CO,18.7,14.2,23.2
273,Minnesota,MN,17.9,13.9,21.9
196,Kansas,KS,17.7,13.4,22.1


### 5. Calculate the average annual growth rates.

Calculate the average annual growth rates (in percent) of overall binge drinking across states for the years the data is available. One way to get these growth rates, is to group the data by state (groupby) and use the first() and last() commands to get the first and last non-NA percentage followed by dividing the calculated percentage increase by the number of years data is available for. Alternatively, you could use the pct_change function to help you out. Provide a table of the 5 states with the largest increases and the 5 states with the largest decreases in binge drinking prevalence over the time period.

In [14]:
# Convert 'binge_all' to numeric
df_wide['binge_all'] = pd.to_numeric(df_wide['binge_all'], errors='coerce')

# Calculate average annual growth rates
def growth_rate(group):
    try:
        years = group.index[-1] - group.index[0]  # Actual years difference
        return (group.iloc[-1] - group.iloc[0]) / group.iloc[0] / years * 100
    except:
        return None

growth_rates = df_wide.groupby('state')['binge_all'].apply(growth_rate)

# Top 5 states with largest increases in binge drinking prevalence
top_5_increases = growth_rates.nlargest(5).reset_index()
top_5_increases.columns = ['State', 'Annual Growth Rate']

# Display label and top 5 states with largest increases
print("Top 5 States with the Largest Increases in Binge Drinking Prevalence:")
display(top_5_increases)

print("\n")  # Adding a newline for separation

# Top 5 states with largest decreases in binge drinking prevalence
top_5_decreases = growth_rates.nsmallest(5).reset_index()
top_5_decreases.columns = ['State', 'Annual Growth Rate']

# Display label and top 5 states with largest decreases
print("Top 5 States with the Largest Decreases in Binge Drinking Prevalence:")
display(top_5_decreases)


Top 5 States with the Largest Increases in Binge Drinking Prevalence:


,State,Annual Growth Rate
0,Tennessee,4.200000
1,West Virginia,1.485149
2,Louisiana,0.683230
3,Kansas,0.411765
4,North Carolina,0.065789




Top 5 States with the Largest Decreases in Binge Drinking Prevalence:


,State,Annual Growth Rate
0,Virgin Islands,-17.105263
1,Delaware,-3.645320
2,Illinois,-3.565217
3,Nevada,-3.172043
4,Oklahoma,-2.787879
